In [1]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install oauth2client

     |████████████████████████████████| 102kB 3.1MB/s ta 0:00:011
     |████████████████████████████████| 102kB 16.9MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

In [2]:
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
]

In [3]:
GOOGLE_KEY_FILE = 'gsheets_key.json'

In [4]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials)

In [5]:
wokbook_key = '1mGcKJNUJJvVbN4ycStlQM6PIyWe7ZX1TSSu0OEjH6eY'
workbook = gc.open_by_key(wokbook_key)

In [6]:
sheet = workbook.get_worksheet(2)
values = sheet.get_all_values()

In [7]:
covid_data = pd.DataFrame(values[1:],columns=values[0])

In [11]:
covid_data.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416,D_20200417
0,Adams County,18001,,,,,1,1,1,1,...,2,2,4,4,5,5,6,6,6,6
1,Allen County,18003,,,,,,,,,...,95,105,119,136,147,156,167,180,224,271
2,Bartholomew County,18005,,,,,,,,,...,66,83,83,108,110,110,110,113,116,130
3,Benton County,18007,,,,,,,,,...,0,0,0,0,2,2,2,3,4,4
4,Blackford County,18009,,,,,,,,,...,4,4,4,5,5,5,6,6,6,6


In [16]:
decatur_johnson_vanderburgh_covid = covid_data.loc[covid_data['County'].isin(['Decatur County', 'Johnson County', 'Vanderburgh County'])]
boone_lake_warrick_covid = covid_data.loc[covid_data['County'].isin(['Boone County', 'Lake County', 'Warrick County'])]
allen_hamilton_hendricks_covid = covid_data.loc[covid_data['County'].isin(['Allen County', 'Hamilton County', 'Hendricks County'])]
marion_monroe_stjoseph_covid = covid_data.loc[covid_data['County'].isin(['Marion County', 'Monroe County', 'St. Joseph County'])]

In [18]:
allen_hamilton_hendricks_covid.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416,D_20200417
1,Allen County,18003,,,,,,,,,...,95,105,119,136,147,156,167,180,224,271
28,Hamilton County,18057,,,,,,,,,...,392,395,431,443,478,490,499,510,520,542
31,Hendricks County,18063,,,1,2,2,2,2,2,...,243,256,287,301,315,330,350,360,378,407


In [32]:
decatur_johnson_vanderburgh_covid.columns

Index(['County', 'GEOGID', 'd_20200306', 'd_20200307', 'd_20200308',
       'd_20200309', 'd_20200310', 'd_20200311', 'd_20200312', 'd_20200313',
       'd_20200314', 'd_20200315', 'd_20200316', 'd_20200317', 'd_20200318',
       'd_20200319', 'd_20200320', 'd_20200321', 'd_20200322', 'D_20200323',
       'D_20200324', 'D_20200325', 'D_20200326', 'D_20200327', 'D_20200328',
       'D_20200329', 'D_20200330', 'D_20200331', 'D_20200401', 'D_20200402',
       'D_20200403', 'D_20200404', 'D_20200405', 'D_20200406', 'D_20200407',
       'D_20200408', 'D_20200409', 'D_20200410', 'D_20200411', 'D_20200412',
       'D_20200413', 'D_20200414', 'D_20200415', 'D_20200416'],
      dtype='object')

In [19]:
djv_melt = decatur_johnson_vanderburgh_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")
blw_melt = boone_lake_warrick_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")
ahh_melt = allen_hamilton_hendricks_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")
mms_melt = marion_monroe_stjoseph_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")

In [20]:
djv_melt['Date'] = djv_melt['Date'].map(lambda x: x.lstrip('dD_'))
blw_melt['Date'] = blw_melt['Date'].map(lambda x: x.lstrip('dD_'))
ahh_melt['Date'] = ahh_melt['Date'].map(lambda x: x.lstrip('dD_'))
mms_melt['Date'] = mms_melt['Date'].map(lambda x: x.lstrip('dD_'))

In [35]:
mms_melt.head()

,County,GEOGID,Date,Count
0,Marion County,18097,20200306,1
1,Monroe County,18105,20200306,
2,St. Joseph County,18141,20200306,
3,Marion County,18097,20200307,1
4,Monroe County,18105,20200307,


In [21]:
import numpy as np
vanderburgh_covid = djv_melt.loc[djv_melt['County'].isin(['Vanderburgh County'])]
vanderburgh_covid = vanderburgh_covid.reset_index(drop=True) 
vanderburgh_covid.replace('', np.nan, inplace=True)
vanderburgh_covid.fillna(0, inplace=True)
vanderburgh_covid['Count'] = vanderburgh_covid['Count'].astype(int)

In [22]:
import pickle
vanderburgh_covid.to_pickle("vanderburgh_covid.pkl")

In [17]:
#vanderburgh_covid.to_json('static/data/vanderburgh_covid.json')

In [23]:
decatur_covid = djv_melt.loc[djv_melt['County'].isin(['Decatur County'])]
decatur_covid = decatur_covid.reset_index(drop=True) 
decatur_covid.replace('', np.nan, inplace=True)
decatur_covid.fillna(0, inplace=True)
decatur_covid['Count'] = decatur_covid['Count'].astype(int)

In [24]:
decatur_covid.to_pickle("decatur_covid.pkl")

In [20]:
#decatur_covid.to_json('static/data/decatur_covid.json')

In [25]:
johnson_covid = djv_melt.loc[djv_melt['County'].isin(['Johnson County'])]
johnson_covid = johnson_covid.reset_index(drop=True) 
johnson_covid.replace('', np.nan, inplace=True)
johnson_covid.fillna(0, inplace=True)
johnson_covid['Count'] = johnson_covid['Count'].astype(int)

In [26]:
johnson_covid.to_pickle("johnson_covid.pkl")

In [23]:
#johnson_covid.to_json('static/data/johnson_covid.json')

In [27]:
Lake_covid = blw_melt.loc[blw_melt['County'].isin(['Lake County'])]
Lake_covid = Lake_covid.reset_index(drop=True) 
Lake_covid.replace('', np.nan, inplace=True)
Lake_covid.fillna(0, inplace=True)
Lake_covid['Count'] = Lake_covid['Count'].astype(int)

In [28]:
Lake_covid.to_pickle("Lake_covid.pkl")

In [29]:
Boone_covid = blw_melt.loc[blw_melt['County'].isin(['Boone County'])]
Boone_covid = Boone_covid.reset_index(drop=True) 
Boone_covid.replace('', np.nan, inplace=True)
Boone_covid.fillna(0, inplace=True)
Boone_covid['Count'] = Boone_covid['Count'].astype(int)

In [30]:
Boone_covid.to_pickle("Boone_covid.pkl")

In [31]:
Warrick_covid = blw_melt.loc[blw_melt['County'].isin(['Warrick County'])]
Warrick_covid = Warrick_covid.reset_index(drop=True) 
Warrick_covid.replace('', np.nan, inplace=True)
Warrick_covid.fillna(0, inplace=True)
Warrick_covid['Count'] = Warrick_covid['Count'].astype(int)

In [32]:
Warrick_covid.to_pickle("Warrick_covid.pkl")

In [39]:
Allen_covid = ahh_melt.loc[ahh_melt['County'].isin(['Allen County'])]
Allen_covid = Allen_covid.reset_index(drop=True) 
Allen_covid.replace('', np.nan, inplace=True)
Allen_covid.fillna(0, inplace=True)
Allen_covid['Count'] = Allen_covid['Count'].astype(int)

Hamilton_covid = ahh_melt.loc[ahh_melt['County'].isin(['Hamilton County'])]
Hamilton_covid = Hamilton_covid.reset_index(drop=True) 
Hamilton_covid.replace('', np.nan, inplace=True)
Hamilton_covid.fillna(0, inplace=True)
Hamilton_covid['Count'] = Hamilton_covid['Count'].astype(int)

Hendricks_covid = ahh_melt.loc[ahh_melt['County'].isin(['Hendricks County'])]
Hendricks_covid = Hendricks_covid.reset_index(drop=True) 
Hendricks_covid.replace('', np.nan, inplace=True)
Hendricks_covid.fillna(0, inplace=True)
Hendricks_covid['Count'] = Hendricks_covid['Count'].astype(int)

Marion_covid = mms_melt.loc[mms_melt['County'].isin(['Marion County'])]
Marion_covid = Marion_covid.reset_index(drop=True) 
Marion_covid.replace('', np.nan, inplace=True)
Marion_covid.fillna(0, inplace=True)
Marion_covid['Count'] = Marion_covid['Count'].astype(int)

Monroe_covid = mms_melt.loc[mms_melt['County'].isin(['Monroe County'])]
Monroe_covid = Marion_covid.reset_index(drop=True) 
Monroe_covid.replace('', np.nan, inplace=True)
Monroe_covid.fillna(0, inplace=True)
Monroe_covid['Count'] = Marion_covid['Count'].astype(int)

StJoseph_covid = mms_melt.loc[mms_melt['County'].isin(['St. Joseph County'])]
StJoseph_covid = StJoseph_covid.reset_index(drop=True) 
StJoseph_covid.replace('', np.nan, inplace=True)
StJoseph_covid.fillna(0, inplace=True)
StJoseph_covid['Count'] = StJoseph_covid['Count'].astype(int)

In [40]:
Allen_covid.to_pickle("Allen_covid.pkl")
Hamilton_covid.to_pickle("Hamilton_covid.pkl")
Hendricks_covid.to_pickle("Hendricks_covid.pkl")

Marion_covid.to_pickle("Marion_covid.pkl")
Monroe_covid.to_pickle("Monroe_covid.pkl")
StJoseph_covid.to_pickle("StJoseph_covid.pkl")

In [42]:
Allen_covid.head(50)

,County,GEOGID,Date,Count
0,Allen County,18003,20200306,0
1,Allen County,18003,20200307,0
2,Allen County,18003,20200308,0
3,Allen County,18003,20200309,0
4,Allen County,18003,20200310,0
5,Allen County,18003,20200311,0
6,Allen County,18003,20200312,0
7,Allen County,18003,20200313,0
8,Allen County,18003,20200314,0
9,Allen County,18003,20200315,0
